In [1]:
import heapq
from collections import namedtuple

import numpy as np

In [2]:
# True
depth = 11991
target_x = 6
target_y = 797

mod = 20183
width = 1000


# Test
# depth = 510
# target_x = 10
# target_y = 10
# mod = 20183
# width = 16


In [3]:
cave_geo = np.zeros((width, width), dtype=int) 
cave_geo[0:, 0] = np.arange(width) * 16807
cave_geo[0, 0:] = np.arange(width) * 48271
cave_erosion = cave_geo.copy()

cave_erosion[0, :] = (cave_geo[0, :] + depth) % mod
cave_erosion[:, 0] = (cave_geo[:, 0] + depth) % mod

for x in range(1, width):
    for y in range(1, width):
        cave_geo[x, y] = (cave_erosion[x - 1, y] * cave_erosion[x, y - 1]) % mod
        cave_geo[0, 0] = 0
        cave_geo[target_x, target_y] = 0
        cave_erosion[x, y] = (cave_geo[x, y] + depth) % mod


In [4]:
cave_type = cave_erosion % 3
cave_type[0, 0] = -1
cave_type[target_x, target_y] = -2

In [5]:
# Only for testing
"""lines = []
for y in range(width):
    line = []
    for x in range(width):
        if cave_type[x, y] == -2:
            line.append('T')
        if cave_type[x, y] == -1:
            line.append('M')
        if cave_type[x, y] == 0:
            line.append('.')
        if cave_type[x, y] == 1:
            line.append('=')
        if cave_type[x, y] == 2:
            line.append('|')
    print(''.join(line))
    lines.append(''.join(line))
"""

"lines = []\nfor y in range(width):\n    line = []\n    for x in range(width):\n        if cave_type[x, y] == -2:\n            line.append('T')\n        if cave_type[x, y] == -1:\n            line.append('M')\n        if cave_type[x, y] == 0:\n            line.append('.')\n        if cave_type[x, y] == 1:\n            line.append('=')\n        if cave_type[x, y] == 2:\n            line.append('|')\n    print(''.join(line))\n    lines.append(''.join(line))\n"

In [6]:
print('Answert to part 1:', cave_type[0:target_x + 1, 0:target_y + 1].sum() + 3)  # Handle my bookeeping negatives

Answert to part 1: 5622


In [7]:
Status = namedtuple('Status', 'dist tool x y')  # Order for sorting

In [8]:
def directions(x, y, width, height):
    if x > 0:
        yield x - 1, y
    if y > 0:
        yield x, y - 1
    if x + 1 < width:
        yield x + 1, y
    if y + 1 < height:
        yield x, y + 1

In [9]:
valid_tools = [['gear', 'torch'], ['gear', 'none'], ['torch', 'none']]

In [10]:
# Make sure source and sink are both rocky
assert (cave_erosion % 3)[0, 0] == 0
assert (cave_erosion % 3)[target_x, target_y] == 0

In [11]:
queue = [Status(dist=0, tool='torch', x=0, y=0), Status(dist=7, tool='gear', x=0, y=0), Status(dist=7, tool='none', x=0, y=0)]
heapq.heapify(queue)
setwise = set()

distances = {tool: -np.ones(cave_type.shape, dtype=int) for tool in ['none', 'torch', 'gear']}
distances['torch'][0, 0] = 0

counter = 0
while True:
    step = heapq.heappop(queue)
    counter += 1
    if counter % 10000 == 0:
        print('Counter {} Distance {}'.format(counter, step.dist))
        
    if distances[step.tool][target_x, target_y] >= 0:
        # We've seen the target
        if distances[tool][target_x, target_y] < step.dist:
            # We've exhausted all paths of lesser length!
            break

    # Try changing equipment
    for tool in valid_tools[max(cave_type[step.x, step.y], 0)]:
        if tool != step.tool:
            new_step = Status(dist=step.dist + 7, tool=tool, x=step.x, y=step.y)
            if new_step in setwise:
                continue
                
            if distances[tool][new_step.x, new_step.y] >= 0:
                if new_step.dist > distances[tool][new_step.x, new_step.y]:
                    continue
            distances[tool][new_step.x, new_step.y] = new_step.dist
            setwise.add(new_step)
            heapq.heappush(queue, new_step)
            
    
    # Step in each direction if possible
    for new_x, new_y in directions(step.x, step.y, width, width):
        can_tools = valid_tools[max(cave_type[new_x, new_y], 0)]
        if step.tool not in can_tools:
            continue
        new_step = Status(dist=step.dist + 1, tool=step.tool, x=new_x, y=new_y)
        if new_step in setwise:
            # Don't go to the same place twice
            continue

        if distances[new_step.tool][new_x, new_y] >= 0:
            # We've been here with this tool
            if new_step.dist > distances[new_step.tool][new_x, new_y]:
                # And we've already gotten here faster
                continue
                    
        setwise.add(new_step)

        distances[new_step.tool][new_x, new_y] = new_step.dist
        heapq.heappush(queue, new_step)

Counter 10000 Distance 106
Counter 20000 Distance 147
Counter 30000 Distance 178
Counter 40000 Distance 204
Counter 50000 Distance 227
Counter 60000 Distance 248
Counter 70000 Distance 268
Counter 80000 Distance 286
Counter 90000 Distance 303
Counter 100000 Distance 319
Counter 110000 Distance 334
Counter 120000 Distance 348
Counter 130000 Distance 362
Counter 140000 Distance 376
Counter 150000 Distance 389
Counter 160000 Distance 401
Counter 170000 Distance 413
Counter 180000 Distance 425
Counter 190000 Distance 437
Counter 200000 Distance 448
Counter 210000 Distance 458
Counter 220000 Distance 469
Counter 230000 Distance 479
Counter 240000 Distance 489
Counter 250000 Distance 499
Counter 260000 Distance 509
Counter 270000 Distance 519
Counter 280000 Distance 528
Counter 290000 Distance 537
Counter 300000 Distance 546
Counter 310000 Distance 555
Counter 320000 Distance 564
Counter 330000 Distance 572
Counter 340000 Distance 581
Counter 350000 Distance 589
Counter 360000 Distance 598
C

In [12]:
min_dist = float('inf')
for tool in ['gear', 'torch', 'none']:
    add_dist = 7 if tool != 'torch' else 0
    poss_dist = distances[tool][target_x, target_y]
    if poss_dist >= 0:
        min_dist = min(min_dist, poss_dist + add_dist)

In [13]:
print('The answer to part 2 is:', min_dist)

The answer to part 2 is: 1089
